In [2]:
import numpy as np
import pandas as pd
import nltk
from nltk.corpus import stopwords
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
import string
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB, GaussianNB
from sklearn.metrics import accuracy_score
from scipy.sparse import csr_matrix
from sklearn.feature_extraction.text import TfidfTransformer
import html
import re
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
from datasets import load_dataset, Dataset
import torch


RuntimeError: Failed to import transformers.trainer because of the following error (look up to see its traceback):
Failed to import transformers.integrations.integration_utils because of the following error (look up to see its traceback):
Failed to import transformers.modeling_tf_utils because of the following error (look up to see its traceback):
Your currently installed version of Keras is Keras 3, but this is not yet supported in Transformers. Please install the backwards-compatible tf-keras package with `pip install tf-keras`.

In [2]:
pip install transformers

Note: you may need to restart the kernel to use updated packages.


  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


Defaulting to user installation because normal site-packages is not writeable
   ---------------------------------------- 0.0/10.4 MB ? eta -:--:--
   ---------------------------------------- 0.0/10.4 MB ? eta -:--:--
   ---------------------------------------- 0.0/10.4 MB 259.2 kB/s eta 0:00:40
   - -------------------------------------- 0.3/10.4 MB 2.0 MB/s eta 0:00:05
   - -------------------------------------- 0.4/10.4 MB 2.3 MB/s eta 0:00:05
   - -------------------------------------- 0.5/10.4 MB 2.5 MB/s eta 0:00:05
   -- ------------------------------------- 0.6/10.4 MB 2.3 MB/s eta 0:00:05
   -- ------------------------------------- 0.7/10.4 MB 2.3 MB/s eta 0:00:05
   --- ------------------------------------ 0.9/10.4 MB 2.4 MB/s eta 0:00:04
   ---- ----------------------------------- 1.1/10.4 MB 2.6 MB/s eta 0:00:04
   ---- ----------------------------------- 1.2/10.4 MB 2.8 MB/s eta 0:00:04
   ----- ---------------------------------- 1.4/10.4 MB 2.8 MB/s eta 0:00:04
   ----- -

In [4]:
pip install datasets

Defaulting to user installation because normal site-packages is not writeableNote: you may need to restart the kernel to use updated packages.

   ---------------------------------------- 0.0/491.4 kB ? eta -:--:--
    --------------------------------------- 10.2/491.4 kB ? eta -:--:--
    --------------------------------------- 10.2/491.4 kB ? eta -:--:--
   -- ------------------------------------ 30.7/491.4 kB 435.7 kB/s eta 0:00:02
   -- ------------------------------------ 30.7/491.4 kB 435.7 kB/s eta 0:00:02
   -- ------------------------------------ 30.7/491.4 kB 435.7 kB/s eta 0:00:02
   -- ------------------------------------ 30.7/491.4 kB 435.7 kB/s eta 0:00:02
   -- ------------------------------------ 30.7/491.4 kB 435.7 kB/s eta 0:00:02
   --- ----------------------------------- 41.0/491.4 kB 103.4 kB/s eta 0:00:05
   --- ----------------------------------- 41.0/491.4 kB 103.4 kB/s eta 0:00:05
   --- ----------------------------------- 41.0/491.4 kB 103.4 kB/s eta 0:00:05
 

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
streamlit 1.32.0 requires packaging<24,>=16.8, but you have packaging 24.1 which is incompatible.



   ------------------------------------- -- 24.3/25.7 MB 2.6 MB/s eta 0:00:01
   -------------------------------------- - 24.4/25.7 MB 2.7 MB/s eta 0:00:01
   -------------------------------------- - 24.5/25.7 MB 2.7 MB/s eta 0:00:01
   -------------------------------------- - 24.6/25.7 MB 2.7 MB/s eta 0:00:01
   -------------------------------------- - 24.7/25.7 MB 2.7 MB/s eta 0:00:01
   -------------------------------------- - 24.9/25.7 MB 2.7 MB/s eta 0:00:01
   -------------------------------------- - 25.0/25.7 MB 2.7 MB/s eta 0:00:01
   ---------------------------------------  25.2/25.7 MB 2.6 MB/s eta 0:00:01
   ---------------------------------------  25.3/25.7 MB 2.6 MB/s eta 0:00:01
   ---------------------------------------  25.5/25.7 MB 2.6 MB/s eta 0:00:01
   ---------------------------------------  25.6/25.7 MB 2.6 MB/s eta 0:00:01
   ---------------------------------------  25.7/25.7 MB 2.6 MB/s eta 0:00:01
   ---------------------------------------  25.7/25.7 MB 2.6 MB

In [2]:
pip install torch

Defaulting to user installation because normal site-packages is not writeable
  Using cached torch-2.7.0-cp312-cp312-win_amd64.whl.metadata (29 kB)
  Using cached sympy-1.14.0-py3-none-any.whl.metadata (12 kB)
   ---------------------------------------- 0.0/212.5 MB ? eta -:--:--
   ---------------------------------------- 0.0/212.5 MB ? eta -:--:--
   ---------------------------------------- 0.0/212.5 MB ? eta -:--:--
   ---------------------------------------- 0.0/212.5 MB ? eta -:--:--
   ---------------------------------------- 0.0/212.5 MB ? eta -:--:--
   ---------------------------------------- 0.0/212.5 MB 217.9 kB/s eta 0:16:16
   ---------------------------------------- 0.0/212.5 MB 217.9 kB/s eta 0:16:16
   ---------------------------------------- 0.0/212.5 MB 217.9 kB/s eta 0:16:16
   ---------------------------------------- 0.0/212.5 MB 217.9 kB/s eta 0:16:16
   ---------------------------------------- 0.0/212.5 MB 217.9 kB/s eta 0:16:16
   --------------------------------

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


In [ ]:
df = pd.read_csv('labeled_data.csv')

In [5]:
df.head()

,Unnamed: 0,count,hate_speech,offensive_language,neither,class,tweet
0,0,3,0,0,3,2,!!! RT @mayasolovely: As a woman you shouldn't...
1,1,3,0,3,0,1,!!!!! RT @mleew17: boy dats cold...tyga dwn ba...
2,2,3,0,3,0,1,!!!!!!! RT @UrKindOfBrand Dawg!!!! RT @80sbaby...
3,3,3,0,2,1,1,!!!!!!!!! RT @C_G_Anderson: @viva_based she lo...
4,4,6,0,6,0,1,!!!!!!!!!!!!! RT @ShenikaRoberts: The shit you...


In [6]:
df.drop(["hate_speech","Unnamed: 0","count","offensive_language","neither"],axis=1,inplace=True)

,class,tweet
0,2,!!! RT @mayasolovely: As a woman you shouldn't...
1,1,!!!!! RT @mleew17: boy dats cold...tyga dwn ba...
2,1,!!!!!!! RT @UrKindOfBrand Dawg!!!! RT @80sbaby...
3,1,!!!!!!!!! RT @C_G_Anderson: @viva_based she lo...
4,1,!!!!!!!!!!!!! RT @ShenikaRoberts: The shit you...


In [8]:
def process(message):
    nopunc = [char for char in message if char not in string.punctuation]
    nopunc = ''.join(nopunc)
    clean = [word for word in nopunc.split() if word.lower() not in stopwords.words('english')]
    
    return ' '.join(clean)

In [9]:
df["tweet"] = df["tweet"].apply(process)

In [10]:
def clean_tweet(tweet):
    # Unescape HTML entities (e.g., &amp; -> &, &#128536; -> emoji)
    tweet = html.unescape(tweet)
    # Remove mentions (@username)
    tweet = re.sub(r'@\w+', '', tweet)
    # Remove hashtags (#hashtag)
    tweet = re.sub(r'#\w+', '', tweet)
    # Remove URLs
    tweet = re.sub(r'http\S+|www\S+|https\S+', '', tweet)
    # Remove emojis and non-ASCII characters
    tweet = tweet.encode('ascii', 'ignore').decode('ascii')
    # Remove special characters, punctuation (except sentence enders), and digits
    tweet = re.sub(r'[^A-Za-z\s]', '', tweet)
    # Collapse multiple spaces into one
    tweet = re.sub(r'\s+', ' ', tweet).strip()
    return tweet

In [11]:
df['tweet'] = df['tweet'].apply(clean_tweet)

In [12]:
porter = nltk.PorterStemmer()
def stem(message):
    stemmedArr = [porter.stem(term) for term in message.split(" ")]
    return ' '.join(stemmedArr)

In [13]:
df["tweet"]=df["tweet"].apply(stem)

In [14]:
df = df.rename(columns={'tweet': 'text','class':'label'})

In [15]:
df.tail()

,label,text
24778,1,you muthafin lie lifeask pearl coreyemanuel ri...
24779,2,youv gone broke wrong heart babi drove redneck...
24780,1,young buck wanna eat dat nigguh like aint fuck...
24781,1,youu got wild bitch tellin lie
24782,2,ruffl ntac eileen dahlia beauti color combin p...


In [17]:
map_dic = {0:"hate_speech",1:"offensive_language",2:"neither"}
df["label"]=df["label"].map(map_dic)

In [18]:
dataset = Dataset.from_pandas(df)

In [20]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

C:\Users\KHALID\AppData\Roaming\Python\Python312\site-packages\huggingface_hub\file_download.py:144: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\KHALID\.cache\huggingface\hub\models--bert-base-uncased. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [21]:
def tokenize_function(example):
    return tokenizer(example["text"], padding="max_length", truncation=True)

In [22]:
tokenized_datasets = dataset.map(tokenize_function, batched=True)

Map:   0%|          | 0/24783 [00:00<?, ? examples/s]

In [23]:
split = tokenized_datasets.train_test_split(test_size=0.3)
train_dataset = split["train"]
eval_dataset = split["test"]

In [24]:
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=3)  # Set correct num_labels

ImportError: 
BertForSequenceClassification requires the PyTorch library but it was not found in your environment.
However, we were able to find a TensorFlow installation. TensorFlow classes begin
with "TF", but are otherwise identically named to our PyTorch classes. This
means that the TF equivalent of the class you tried to import would be "TFBertForSequenceClassification".
If you want to use TensorFlow, please use TF classes instead!

If you really do want to use PyTorch please go to
https://pytorch.org/get-started/locally/ and follow the instructions that
match your environment.
